# Worksheet 08

Name:  Nina Athma
UID: U51803274

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[6.809238874203322, 3.8430318566353465, 5.168648488749895, 4.029150161594263, 4.419003511189607, 5.545487489136424, 5.952450118277096, 5.031586489479561, 5.573940614380515, 4.39564365543242]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [3]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[7.264301574636394, 8.495149752892523, 8.450528967201775, 8.426332126557252, 10.10401539172487, 9.526352911163631, 6.621217119058324, 7.455550887916708, 8.758207619275794, 7.831347885733749]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [4]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1= s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[7.831347885733749, 8.758207619275794, 4.39564365543242, 7.455550887916708, 5.573940614380515, 5.031586489479561, 6.621217119058324, 5.952450118277096, 9.526352911163631, 5.545487489136424]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Mean and variance describe the distribution of each cluster.

s1_mean = 5, s1_var = 1

s2_mean=8, s2_var=1

cardinality = 10

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [12]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2)/(len(s1)+len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.39564365543242, 5.573940614380515, 5.031586489479561, 6.621217119058324, 5.952450118277096, 5.545487489136424]
[7.831347885733749, 8.758207619275794, 7.455550887916708, 9.526352911163631]
P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 5.52005424762739,  mean_2 = 8.392864826022471
var_1 = 0.48432939654310925,  var_2 = 0.6530323221344142


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [15]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0]*prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1]*prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  7.831347885733749
probability of observing that point if it came from cluster 0 =  9.345017604695026e-06
probability of observing that point if it came from cluster 1 =  0.4221096191803017
point =  8.758207619275794
probability of observing that point if it came from cluster 0 =  1.6186812809443532e-10
probability of observing that point if it came from cluster 1 =  0.5224085476727631
point =  4.39564365543242
probability of observing that point if it came from cluster 0 =  0.0556417828466975
probability of observing that point if it came from cluster 1 =  4.468436903564918e-09
point =  7.455550887916708
probability of observing that point if it came from cluster 0 =  0.0002805056574300794
probability of observing that point if it came from cluster 1 =  0.2180808284776517
point =  5.573940614380515
probability of observing that point if it came from cluster 0 =  0.8186178852158207
probability of observing that point if it came from cluster 1 =  5.4920471671306494e-05
point =  

We can assume that each point x belongs to the cluster for which prob_Si_x is greatest.

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [18]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [ sum(prob_s0_x)/len(prob_s0_x) , sum(prob_s1_x)/len(prob_s1_x) ]

print("P(S_1) = " + str(prob_c[0]) + ",  P(S_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.5859317755501537,  P(S_2) = 0.4140682244498463
mean_1 = 5.493989492151155,  mean_2 = 8.33214252453051
var_1 = 0.5859317755501537,  var_2 = 0.4140682244498463


The values are pretty close to those from Kmeans.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)